<a href="https://colab.research.google.com/github/abergues/SalsaApp/blob/eero/Exploratory_analysis_of_op.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exporing the Annotation from openpose

There are several properties of the data that should be checked:
1. Missing values
  * Are the patterns, holes
2. Jumps in the data
3. Patterns in the data
  * Feet - beat, speed, possible velosity, steps
  * Hips - beat, movement, direction, turn



In [ ]:
import sys
!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

# libraries needed from outside
import os
from os import mkdir
from os.path import exists, join, basename, splitext
import sys
import glob                       # help for finding files
from re import split              # regular expression string splitter
import subprocess                 #subprocess wraps around regular os commands
import shutil                     # used for copying files in the os.
import pandas as pd

     |████████████████████████████████| 129 kB 5.1 MB/s 
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:


from google.colab import drive
drive.mount('/content/gdrive')
 
if os.path.isdir('/content/gdrive/MyDrive'):     # Ana
  google_drive_path = '/content/gdrive/MyDrive'
if os.path.isdir('/content/gdrive/MyDrive/DSR'):  # Eero. ipython can deal with MyDrive, 
  # but some bash commands require the space in "My Drive" to work.  and some fail.
  # I believe this has to do with what kind of " " '' one is using, as they must not be mixed. 
  google_drive_path = '/content/gdrive/MyDrive/DSR' 
else:
  print("There is a problem google_drive_path.")

root_path   = os.path.join(google_drive_path, "DanceApp")  
analysis_path = os.path.join(google_drive_path, "DanceApp","exploratory")  
input_data_path = os.path.join(google_drive_path, "DanceApp","output_op_1p" )  
script_path = "/content/gdrive/MyDrive/DSR/2021-DSR-Porfolio/python_scripts"
#one cannot add items to sys.path that have a space in them and script_path must be
#the same for Ana and Eero.  Eero has now added a link from his google DSR to Ana's google 2021-DSR-Porfolio/

print("Using ", root_path, " as root_path")

if not os.path.exists(root_path): 
    os.mkdir(root_path)


def check_path(x):
  if os.path.isdir(x):
    print("Using", x )
  else:
    print("Problem: There was no ", x)  # No error keeps running

def stop_if_no_path(x):
  if os.path.isdir(x):
    print("Using", x )
  else:
    raise Exception("Problem: There was no ", x)  # Throws and error and stops

check_path(root_path)
check_path(input_data_path)
check_path(analysis_path)



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Using  /content/gdrive/MyDrive/DSR/DanceApp  as root_path
Using /content/gdrive/MyDrive/DSR/DanceApp
Using /content/gdrive/MyDrive/DSR/DanceApp/output_op_1p
Using /content/gdrive/MyDrive/DSR/DanceApp/exploratory


#Looking at one clip at the time

In [ ]:
print(os.listdir(input_data_path)) # use these to select clip_names

['1p_Artur_05122021_chreo1', '1p_Earl_05122021_choreo1_flipped_lr', '1p_Eero_1_05122021_choreo1', '1p_Eero_4_05122021_choreo1', '1p_Eero_6_05122021_choreo1', '1p_Eero_7_05122021_choreo1', '1p_Dirk_choreo1_04.12.2021', '1P-Person1-whole', '1p_Raquelinha_05122021_choreo1', '1p_sarah_salsa_basics_choreo1', '1p_Stef_1_05122021_Choreo1', '1p_Stef_2_05122021_Choreo1', '1p_Stef_3_05122021_chroeo1', '1p_Heidi_07122021_choreo1_-turns', '1p_JonathanB_05122021_choreo1', 'Data_concat_cut.csv', 'Data_concat.csv']


In [ ]:
clip_name = '1p_Raquelinha_05122021_choreo1'
 # "1p_Eero_7_05122021_choreo1" 
 # "1p_Dirk_choreo1_04.12.2021"
print("clip_name: ", clip_name)

check_path(root_path)


input_csv = os.path.join(input_data_path, clip_name, "Data.csv")

if os.path.isfile(input_csv):
  print("Found the csv")
else:
  print("The , ", input_csv, " file does not exist.")
  # raise Exception("No input csv file")

data_full = pd.read_csv(input_csv)
data_full

clip_name:  1p_Raquelinha_05122021_choreo1
Using /content/gdrive/MyDrive/DSR/DanceApp
Found the csv


,clip_name,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,lear_y,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,label,status
0,1p_Raquelinha_05122021_choreo1,0,0,0.0,0.0,294.2950,40.6758,168.72100,45.6680,80.8513,211.495,123.505,28.06270,417.325,30.6657,515.281,201.366,0.000,0.000,349.469,492.674,271.6960,502.691,332.015,781.414,0.00,0.00,419.857,485.117,427.310,753.782,417.344,811.514,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,454.957,854.225,470.020,846.672,414.785,811.590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.477907,0.289934,0.586650,0.204628,0.401722,0.163936,0.000000,0.532024,0.490697,0.551323,0.000000,0.459675,0.328568,0.319683,0.0,0.0,0.0,0.0,0.304639,0.306539,0.216581,0.0,0.0,0.0,NaN,NaN
1,1p_Raquelinha_05122021_choreo1,1,0,0.0,0.0,309.3240,85.7944,206.36900,98.4012,75.8814,171.344,136.109,6.70860,404.761,68.2620,0.000,0.000,0.000,0.000,347.040,495.124,271.6890,505.189,331.915,786.374,0.00,0.00,414.856,487.584,414.856,718.627,397.207,813.961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,467.449,851.683,470.134,841.626,379.682,826.620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.614462,0.520312,0.689418,0.306733,0.423187,0.000000,0.000000,0.527197,0.495980,0.541786,0.000000,0.473679,0.357710,0.593638,0.0,0.0,0.0,0.0,0.607733,0.578404,0.301699,0.0,0.0,0.0,NaN,NaN
2,1p_Raquelinha_05122021_choreo1,2,0,0.0,0.0,316.9150,90.9845,211.43600,108.5130,80.8328,148.588,148.574,6.70905,422.351,83.3397,0.000,0.000,0.000,0.000,349.394,495.171,271.7420,507.687,331.944,778.860,0.00,0.00,414.830,487.584,409.805,731.221,392.170,813.988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,467.560,849.214,470.049,836.669,372.235,831.575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.593271,0.524648,0.703206,0.297726,0.403010,0.000000,0.000000,0.540486,0.498197,0.580319,0.000000,0.494412,0.529284,0.685670,0.0,0.0,0.0,0.0,0.649105,0.631212,0.382857,0.0,0.0,0.0,NaN,NaN
3,1p_Raquelinha_05122021_choreo1,3,0,0.0,0.0,316.8650,103.4350,213.96500,110.9500,90.9474,133.566,158.653,6.71863,424.808,90.9698,515.284,256.619,0.000,0.000,342.005,492.684,271.6880,502.681,332.003,776.381,0.00,0.00,409.784,487.597,397.212,751.265,389.736,814.089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,472.484,844.160,470.112,834.171,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.638005,0.556388,0.727547,0.235978,0.488131,0.096439,0.000000,0.525312,0.500404,0.565277,0.000000,0.531347,0.519205,0.359207,0.0,0.0,0.0,0.0,0.389454,0.368969,0.000000,0.0,0.0,0.0,NaN,NaN
4,1p_Raquelinha_05122021_choreo1,4,0,0.0,0.0,319.3600,110.9840,221.47700,126.0280,105.9510,143.639,161.181,6.70403,417.324,98.4511,515.314,198.865,467.533,108.466,341.972,500.152,271.7750,507.691,334.412,773.845,0.00,0.00,409.682,490.174,392.141,736.252,389.585,806.517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,470.025,841.649,467.530,834.162,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.691527,0.583764,0.732657,0.381050,0.504977,0.180731,0.153231,0.513343,0.488512,0.536536,0.000000,0.520855,0.533638,0.091153,0.0,0.0,0.0,0.0,0.227484,0.205679,0.000000,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3277,1p_Raquelinha_05122021_choreo1,3277,0,0.0,0.0,68.2890,83.3478,0.00000,0.0000,0.0000,0.000,0.000,0.00000,208.850,105.9610,281.711,316.847,331.834,467.500,171.290,467.530,103.419

In [40]:
data_cut = data_full.iloc[100:400,:]
#len(data.columns.unique()) # all column names are unique
data_cut

,clip_name,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,lear_y,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,label,status
100,1p_Artur_05122021_chreo1,350,0.0,269.127,191.246,266.454,286.607,171.140,279.044,123.4750,379.473,131.080,472.274,354.377,289.101,391.924,389.595,354.357,452.235,254.082,522.482,208.677,520.038,206.287,670.675,208.841,793.634,309.202,522.551,294.179,680.621,286.590,831.189,251.556,171.314,289.103,176.148,231.335,186.216,309.233,191.174,289.024,891.527,304.173,886.464,271.546,836.287,181.2380,843.788,168.6890,833.668,221.326,806.122,0.942220,0.818825,0.779502,0.782900,0.904086,0.798293,0.901972,0.884112,0.651284,0.640344,0.785916,0.851661,0.675125,0.714845,0.790381,0.925927,0.917039,0.893723,0.943726,0.721839,0.657257,0.581931,0.740861,0.785418,0.653069,basic,NaN
101,1p_Artur_05122021_chreo1,351,0.0,269.102,191.292,266.458,286.623,171.242,279.095,125.9160,384.446,136.003,469.848,351.975,289.061,389.445,391.994,356.859,457.278,256.446,514.961,206.229,515.001,206.295,670.651,208.880,793.534,311.615,515.036,309.057,655.607,286.521,818.696,251.494,173.688,289.045,176.203,228.982,188.697,309.183,191.208,289.087,891.302,306.655,878.861,271.466,828.725,181.2440,848.690,168.7020,833.772,223.873,803.582,0.940174,0.815680,0.771326,0.811794,0.879123,0.801963,0.903698,0.827973,0.679856,0.653969,0.774647,0.855031,0.669869,0.774271,0.772098,0.930329,0.904378,0.890774,0.959317,0.753533,0.686391,0.689697,0.765579,0.775498,0.676616,basic,NaN
102,1p_Artur_05122021_chreo1,352,0.0,268.999,191.360,266.431,288.985,171.272,281.579,126.0040,376.955,143.562,454.788,351.929,289.118,386.903,391.986,359.429,467.236,254.025,512.545,206.272,512.565,206.315,670.671,208.899,793.514,309.185,514.969,309.152,653.127,279.066,801.101,251.422,173.734,286.644,178.667,228.976,188.642,309.146,191.167,289.057,868.788,304.181,856.334,268.970,808.612,181.1780,848.791,168.6720,836.186,226.303,801.056,0.927266,0.820317,0.769992,0.824918,0.839247,0.804344,0.851773,0.844377,0.703097,0.673992,0.746581,0.830958,0.702785,0.828668,0.769732,0.933484,0.876304,0.883334,0.962439,0.790662,0.734362,0.703025,0.765126,0.765110,0.704543,basic,NaN
103,1p_Artur_05122021_chreo1,353,0.0,269.011,191.358,266.511,289.054,173.749,284.076,128.4470,381.962,151.124,459.748,351.921,291.521,389.421,392.091,364.409,469.715,253.977,515.008,206.292,514.980,208.696,665.583,211.238,791.108,309.137,517.503,309.190,652.958,274.038,776.050,251.421,176.152,286.643,178.739,228.984,188.821,309.090,191.306,284.039,838.753,296.670,831.290,261.474,781.048,181.2070,851.176,168.6880,836.232,226.296,796.043,0.938408,0.843890,0.762881,0.828579,0.829263,0.803926,0.888503,0.881108,0.697704,0.663909,0.746468,0.847279,0.695625,0.867244,0.807811,0.925029,0.870424,0.913629,0.944408,0.751844,0.729585,0.640853,0.756648,0.738432,0.731467,basic,NaN
104,1p_Artur_05122021_chreo1,354,0.0,268.973,191.354,268.984,289.084,181.158,286.541,130.9710,389.410,161.104,452.261,354.327,289.184,389.581,392.081,366.835,469.747,251.548,512.424,206.240,510.021,208.698,663.039,208.872,791.069,306.644,512.483,309.088,650.394,271.455,758.478,251.387,176.209,286.605,181.167,231.280,191.207,309.074,191.370,276.573,813.735,291.762,808.628,254.026,760.920,181.1920,848.829,168.6810,836.207,218.824,796.060,0.933400,0.881572,0.754869,0.895786,0.828352,0.798790,0.907088,0.895502,0.718558,0.662081,0.752510,0.872224,0.681881,0.835435,0.818245,0.930820,0.870456,0.89177

In [ ]:
# detect dict or list columns
# list_or_dict = data.applymap(lambda x: isinstance(x, dict) or isinstance(x, list)).all()
# list_or_dict

### Pandas Profiling
This works but is awfully heavy and the resulting HTML are 250MB each.

In [ ]:
# ! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

# from pandas_profiling import ProfileReport 

In [ ]:
# # https://python.plainenglish.io/how-to-use-pandas-profiling-on-google-colab-e34f34ff1c9f
# reportname = clip_name + "_profile"
# print(reportname)
# profile = ProfileReport(data_t)
# # profile.to_notebook_iframe()  # supposedly working in Colab, but not really
# # profile.to_widgets()          # not supported by Colab
# profile.to_file(analysis_path + "/" + reportname + ".html")

1p_Raquelinha_05122021_choreo1_profile


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(analysis_path + "/" + reportname + ".html")
os.getcwd()

/content/gdrive/MyDrive/DSR/DanceApp/exploratory/1p_Raquelinha_05122021_choreo1_profile.html


'/content'

# Looking at the full data frame with all the clips

In [ ]:

input_csv = os.path.join(input_data_path, "Data_concat_cut.csv")

if os.path.isfile(input_csv):
  print("Found the csv")
else:
  print("The , ", input_csv, " file does not exist.")
  # raise Exception("No input csv file")

data_full = pd.read_csv(input_csv)
data_full

In [63]:
import plotly as py
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
# !pip install orca  # needed to crate and save image files.
# py.io.orca.config.executable = '/usr/local/lib/python3.7/dist-packages/orca/orca'

df = data_full # .query("clip_name=='1p_Artur_05122021_chreo1'")

image_folder = os.path.join(analysis_path, "images")
if not os.path.exists(image_folder):
    os.mkdir(image_folder)

In [65]:
fig = px.line(df, x="lshoulder_y", y="rshoulder_y", color = "clip_name")
fig.update_layout(title="Comparing L&R shoulder heights")
fig.show()


In [66]:
fig = px.line(df, x="frame_nr", y="neck_y", color = "clip_name")
fig.update_layout(title="Neck heights seem low for all")
fig.show()

In [68]:


fig = px.line(df, x="frame_nr", y="neck_x", color = "clip_name")
fig.update_layout(title="Comparing neck on X-axis")
fig.show()



In [73]:
fig = px.line(df, x="lshoulder_y" , y="midhip_y", color = "clip_name")
fig.update_layout(title="Hips are higher on Y than shoulders")
fig.show()

In [ ]:
from pandas.plotting import autocorrelation_plot

plt.figure();

data = pd.Series(data_t['lknee_x'])

autocorrelation_plot(data);

Found the csv


,clip_name,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,lear_y,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,label,status
0,1p_Artur_05122021_chreo1,250,0.0,249.061,196.239,251.496,286.575,168.695,284.129,126.032,384.505,148.487,462.319,334.373,286.517,389.451,371.994,369.318,452.317,233.854,517.516,188.704,514.977,191.351,653.133,208.770,755.953,286.518,522.533,269.103,670.503,266.531,808.631,233.885,183.739,268.928,186.202,213.821,191.248,289.004,191.356,271.570,866.345,289.120,856.301,251.578,813.622,178.736,801.102,168.740,791.141,218.868,768.462,0.905253,0.882538,0.807618,0.828615,0.842831,0.782190,0.897228,0.907502,0.684465,0.682405,0.765624,0.853827,0.642654,0.810555,0.800686,0.884752,0.905192,0.898377,0.947265,0.790894,0.773175,0.647946,0.812466,0.850178,0.733774,basic,S
1,1p_Artur_05122021_chreo1,251,0.0,251.454,193.828,253.934,286.640,171.081,286.549,128.490,387.022,148.444,472.305,339.307,286.568,389.456,372.045,369.356,452.351,233.923,522.531,188.778,519.996,191.333,658.076,208.813,755.937,286.537,527.502,271.519,670.562,266.574,808.607,236.369,183.680,269.057,186.178,213.917,191.247,289.071,193.726,271.540,866.376,289.111,856.336,253.943,813.596,178.727,801.079,168.740,791.118,221.373,768.420,0.909779,0.866953,0.801844,0.877927,0.893125,0.780592,0.889844,0.910966,0.677615,0.683593,0.755941,0.854289,0.657238,0.834463,0.810987,0.856459,0.921658,0.875690,0.921918,0.806627,0.771766,0.630645,0.804178,0.840649,0.729915,basic,NaN
2,1p_Artur_05122021_chreo1,252,0.0,251.543,193.853,256.475,286.643,171.284,286.523,130.914,389.578,146.026,474.930,341.840,286.568,386.967,376.950,369.418,452.332,233.980,524.982,188.789,519.975,191.293,655.598,208.836,753.527,286.649,527.570,271.582,670.508,268.902,808.624,238.904,183.690,271.426,186.184,216.377,191.224,289.230,193.728,271.552,866.347,289.145,856.328,253.951,813.641,181.233,801.072,171.078,791.118,223.806,763.435,0.902302,0.842373,0.788181,0.910095,0.841959,0.780050,0.828608,0.924542,0.677021,0.691645,0.772269,0.862838,0.669216,0.815470,0.820885,0.841014,0.930492,0.882195,0.943283,0.803007,0.774799,0.633506,0.810073,0.833977,0.715198,basic,NaN
3,1p_Artur_05122021_chreo1,253,0.0,251.599,193.823,256.469,289.012,171.270,286.601,130.954,391.994,140.991,484.843,341.833,286.650,379.485,381.974,369.350,452.222,248.854,514.964,191.360,512.534,191.321,650.550,211.233,753.378,294.155,517.509,271.664,660.587,268.916,808.669,241.393,183.643,271.497,186.177,218.819,191.248,291.606,193.763,271.571,868.768,289.149,856.368,253.975,813.664,183.690,798.580,171.104,791.055,226.367,758.423,0.907760,0.847579,0.795651,0.868299,0.804663,0.782194,0.789320,0.936142,0.680573,0.677474,0.827008,0.879269,0.637561,0.762815,0.829053,0.837988,0.939281,0.876877,0.928143,0.802044,0.771636,0.628547,0.823398,0.837598,0.705927,basic,NaN
4,1p_Artur_05122021_chreo1,254,0.0,254.001,193.773,256.494,288.984,173.675,286.574,131.010,389.549,133.604,487.394,341.859,286.651,374.475,384.457,366.911,449.773,248.989,512.556,193.807,512.473,191.361,650.609,208.921,753.368,299.170,515.026,271.680,668.030,268.936,811.017,243.897,181.247,271.577,186.176,218.845,191.326,291.647,193.817,271.553,866.369,289.140,856.359,253.971,813.684,186.145,798.575,171.139,791.058,226.299,758.394,0.910799,0.842652,0.780982,0.877818,0.854954,0.777401,0.804724,0.914783,0.702386,0.657017,0.811490,0.884676,0.631724,0.772847,0.837267,0.851566,0.935580,0.841408,0.914674,0.800491,0.77497

In [ ]:
data_full_by_person = data_full.groupby(by="clip_name")
data_full_by_person.

In [ ]:
data_full_by_person.describe()

frame_nr          ...   rheal_v          
                                       count    mean  ...       75%       max
clip_name                                             ...                    
1P-Person1-whole                      3676.0  1898.5  ...  0.804169  0.972614
1p_Artur_05122021_chreo1              3680.0  2089.5  ...  0.745644  0.928022
1p_Dirk_choreo1_04.12.2021            3674.0  2189.5  ...  0.682817  0.858777
1p_Earl_05122021_choreo1_flipped_lr   3099.0  1586.0  ...  0.820720  0.957255
1p_Eero_1_05122021_choreo1            3602.0  2074.5  ...  0.787546  0.965117
1p_Eero_4_05122021_choreo1            3599.0  2135.0  ...  0.784822  0.909701
1p_Eero_6_05122021_choreo1            3600.0  1942.5  ...  0.785422  0.905784
1p_Eero_7_05122021_choreo1            3601.0  2085.0  ...  0.804131  0.915242
1p_Heidi_07122021_choreo1_-turns      6194.0  3454.5  ...  0.756383  0.908724
1p_Raquelinha_05122021_choreo1        2824.0  1583.5  ...  0.727502  0.859791
1p_Stef_2_05122021_Choreo1            3596.0  1914.5  ...  0.821941  0.959599
1p_sarah_salsa_basics_choreo1         3307.0  1713.0  ...  0.566263  0.855356

[12 rows x 616 columns]

In [27]:
data_full[data_full.clip_name=="1p_Stef_2_05122021_Choreo1"]

,clip_name,frame_nr,person_id,nose_x,nose_y,neck_x,neck_y,rshoulder_x,rshoulder_y,relbow_x,relbow_y,rwrist_x,rwrist_y,lshoulder_x,lshoulder_y,lelbow_x,lelbow_y,lwrist_x,lwrist_y,midhip_x,midhip_y,rhip_x,rhip_y,rknee_x,rknee_y,rankle_x,rankle_y,lhip_x,lhip_y,lknee_x,lknee_y,lankle_x,lankle_y,reye_x,reye_y,leye_x,leye_y,rear_x,rear_y,lear_x,lear_y,lbigtoe_x,lbigtoe_y,lsmalltoe_x,lsmalltoe_y,lheal_x,lheal_y,rbigtoe_x,rbigtoe_y,rsmalltoe_x,rsmalltoe_y,rheal_x,rheal_y,nose_v,neck_v,rshoulder_v,relbow_v,rwrist_v,lshoulder_v,lelbow_v,lwrist_v,midhip_v,rhip_v,rknee_v,rankle_v,lhip_v,lknee_v,lankle_v,reye_v,leye_v,rear_v,lear_v,lbigtoe_v,lsmalltoe_v,lheal_v,rbigtoe_v,rsmalltoe_v,rheal_v,label,status
40856,1p_Stef_2_05122021_Choreo1,117,0.0,241.413,286.582,246.371,349.464,191.308,351.746,171.268,429.618,208.718,437.227,291.569,349.417,319.206,412.103,301.668,447.209,238.879,492.440,208.824,492.431,213.815,612.956,228.897,713.361,269.152,492.465,271.553,625.443,274.144,770.862,231.280,271.683,251.411,271.648,216.336,286.628,268.977,286.638,281.543,811.165,294.172,808.552,271.469,775.986,226.424,750.934,211.333,745.924,231.430,723.247,0.860948,0.921365,0.928600,0.929301,0.877481,0.939121,0.871441,0.853124,0.771416,0.806781,0.831455,0.902300,0.817864,0.841824,0.785578,0.948035,0.959754,0.837866,0.928386,0.815457,0.793880,0.733758,0.888604,0.856727,0.714746,basic,S
40857,1p_Stef_2_05122021_Choreo1,118,0.0,246.406,286.574,246.459,349.460,193.699,349.461,171.308,429.598,208.724,437.211,291.679,349.425,319.219,412.060,309.084,447.213,241.440,494.904,208.900,494.894,213.890,615.386,228.921,713.313,271.553,494.931,271.640,632.898,274.124,771.021,231.448,274.008,251.564,271.666,221.323,286.631,269.070,286.625,281.570,813.544,294.171,808.619,271.482,781.004,228.766,750.888,211.349,745.863,231.467,718.318,0.888633,0.937983,0.917794,0.918350,0.882079,0.937207,0.880552,0.908329,0.764076,0.801042,0.821549,0.908350,0.802178,0.851437,0.810785,0.933834,0.936867,0.850419,0.922553,0.836454,0.846876,0.720565,0.875752,0.847424,0.700515,basic,NaN
40858,1p_Stef_2_05122021_Choreo1,119,0.0,248.884,286.518,248.845,349.458,193.785,349.437,173.692,429.532,208.780,442.211,294.106,349.461,321.751,412.173,309.229,447.167,243.967,494.953,211.283,494.912,216.354,615.418,228.912,713.314,271.613,497.382,274.019,630.489,274.107,770.886,233.910,271.648,254.058,271.596,223.815,286.545,271.481,284.093,281.593,811.223,294.164,808.584,269.121,776.060,226.386,750.873,211.305,743.421,231.438,720.745,0.904128,0.950924,0.907763,0.891599,0.870623,0.920269,0.857963,0.896812,0.763343,0.795233,0.790929,0.906113,0.783334,0.854189,0.791847,0.918646,0.912537,0.848468,0.881030,0.830910,0.822245,0.709360,0.855990,0.832608,0.686339,basic,NaN
40859,1p_Stef_2_05122021_Choreo1,120,0.0,248.967,284.057,248.903,349.427,196.225,349.396,173.681,427.147,208.714,444.718,294.182,349.430,324.227,412.215,309.285,447.138,246.414,492.447,211.352,492.392,216.301,615.379,228.982,713.238,271.682,492.510,274.066,630.424,274.123,770.942,236.385,271.582,256.536,271.572,226.274,286.555,271.538,286.560,281.564,813.559,294.145,808.631,271.492,778.522,226.323,750.898,211.281,743.406,233.949,715.809,0.896623,0.959314,0.895275,0.887795,0.875776,0.912721,0.854584,0.890267,0.805770,0.819050,0.812165,0.916876,0.792466,0.823710,0.795493,0.904880,0.893498,0.871229,0.910937,0.831064,0.836970,0.705293,0.851304,0.816040,0.690853,basic,NaN
40860,1p_Stef_2_05122021_Choreo1,121,0.0,249.048,281.569,248.953,349.376,196.287,349.334,173.631,424.595,206.237,442.235,296.613,349.402,326.688,414.670,309.269,444.747,246.496,492.497,211.412,492.444,216.349,622.908,231.278,713.218,274.084,494.909,274.112,632.880,274.148,770.965,238.905,271.487,261.477,271.528,226.365,284.083,271.599,286.551,281.538,813.533,294.183,808.631,271.487,778.521,226.365,750.987,211.308,745.898,233.985,715.729,0.895692,0.968046,0.883198,0.870169,0.849505,0.898924,0.854619,0.878169,0.796048,0.793711,0.781978,0.918038,0.768547,0.816998,0.804595,0.902331,0.891397,0.87